In [1]:
import subprocess
import sys
import os
from pathlib import Path
import shutil

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# 0) Hard reset any prior torch stacks
for pkg in ('torch', 'torchvision', 'torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean stray site dirs
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

# 1) Install the EXACT cu121 torch stack FIRST
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# 2) Create constraints file
Path('constraints.txt').write_text(
    'torch==2.4.1\n'
    'torchvision==0.19.1\n'
    'torchaudio==2.4.1\n'
)

# 3) Install NON-torch deps
pip('install', '-c', 'constraints.txt',
    'transformers==4.44.2', 'accelerate==0.34.2',
    'datasets==2.21.0', 'evaluate==0.4.2',
    'sentencepiece', 'scikit-learn',
    '--upgrade-strategy', 'only-if-needed')

# 4) Additional packages for this task
pip('install', 'nibabel', 'SimpleITK', 'timm', 'albumentations', 'scikit-image')

# 5) Sanity check
import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version, 'cuda', '')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0))

# Test imports for preprocessing
import nibabel as nib
import SimpleITK as sitk
import timm
import albumentations as A
from skimage import transform
print('All packages imported successfully')

> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 533.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 473.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 492.9 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 468.5 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.4.1+cu121 requires torch==2.4.1, but you have torch 2.8.0 which is incompatible.
datasets 2.21.0 requires fsspec[http]<=2024.6.1,>=2023.1.0, but you have fsspec 2025.9.0 which is incompatible.


torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q


/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All packages imported successfully


In [5]:
import subprocess
import os
result = subprocess.run(['python3', 'scripts/mip_prep.py'], check=True, capture_output=False)
print('Preprocessing completed successfully.' if result.returncode == 0 else 'Error occurred.')

# Verify sample outputs
import numpy as np
import pandas as pd

if os.path.exists('data/train_mips.csv'):
    train_mips = pd.read_csv('data/train_mips.csv')
    print(f'Train MIPs shape: {train_mips.shape}')
    print(train_mips.head())

sample_train_npy = 'data/mips/train/' + train_mips['StudyInstanceUID'].iloc[0] + '.npy'
if os.path.exists(sample_train_npy):
    sample = np.load(sample_train_npy)
    print(f'Sample MIP shape: {sample.shape}')
    print(f'Sample min: {sample.min():.3f}, max: {sample.max():.3f}, mean: {sample.mean():.3f}')

if os.path.exists('data/test_mips_sub.csv'):
    test_mips_sub = pd.read_csv('data/test_mips_sub.csv')
    print(f'Test subsample MIPs shape: {test_mips_sub.shape}')

ImageSeriesReader (0x30e22050): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.0005



ImageSeriesReader (0x30e22050): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.0005



ImageSeriesReader (0x30e22050): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000996885



ImageSeriesReader (0x30e22050): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.0005



ImageSeriesReader (0x30e22050): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.0005



...

ImageSeriesReader (0x30e22050): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.0005



ImageSeriesReader (0x30e22050): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.0005



ImageSeriesReader (0x30e22050): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000996885



ImageSeriesReader (0x30e22050): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000997319



Processed train 1.2.826.0.1.3680043.25891, MIP shape: (3, 384, 384)
Processed train 1.2.826.0.1.3680043.25548, MIP shape: (3, 384, 384)
Processed train 1.2.826.0.1.3680043.28978, MIP shape: (3, 384, 384)
Processed train 1.2.826.0.1.3680043.9904, MIP shape: (3, 384, 384)
Processed 120 train volumes
Processed train 1.2.826.0.1.3680043.5778, MIP shape: (3, 384, 384)
Processed train 1.2.826.0.1.3680043.26801, MIP shape: (3, 384, 384)
Processed train 1.2.826.0.1.3680043.9028, MIP shape: (3, 384, 384)
Processed train 1.2.826.0.1.3680043.3820, MIP shape: (3, 384, 384)
Processed train 1.2.826.0.1.3680043.7209, MIP shape: (3, 384, 384)
Processed train 1.2.826.0.1.3680043.24562, MIP shape: (3, 384, 384)
Processed train 1.2.826.0.1.3680043.1107, MIP shape: (3, 384, 384)
Processed train 1.2.826.0.1.3680043.9506, MIP shape: (3, 384, 384)
Processed train 1.2.826.0.1.3680043.32280, MIP shape: (3, 384, 384)
Processed train 1.2.826.0.1.3680043.17194, MIP shape: (3, 384, 384)
Processed 130 train volumes

In [4]:
import os
print('Number of items in train_images:', len(os.listdir('train_images')))
print('First 5 items in train_images:', os.listdir('train_images')[:5])
print('Number of items in test_images:', len(os.listdir('test_images')))
print('First 5 items in test_images:', os.listdir('test_images')[:5])

# Check if subdirs or files
for item in os.listdir('train_images')[:3]:
    path = f'train_images/{item}'
    print(f'{item}: isdir={os.path.isdir(path)}, isfile={os.path.isfile(path)}, size={os.path.getsize(path) if os.path.isfile(path) else "N/A"}')

for item in os.listdir('test_images')[:3]:
    path = f'test_images/{item}'
    print(f'{item}: isdir={os.path.isdir(path)}, isfile={os.path.isfile(path)}')

Number of items in train_images: 202
First 5 items in train_images: ['1.2.826.0.1.3680043.30020', '1.2.826.0.1.3680043.4740', '1.2.826.0.1.3680043.30640', '1.2.826.0.1.3680043.24617', '1.2.826.0.1.3680043.15485']
Number of items in test_images: 1817
First 5 items in test_images: ['1.2.826.0.1.3680043.5581', '1.2.826.0.1.3680043.7479', '1.2.826.0.1.3680043.26665', '1.2.826.0.1.3680043.6836', '1.2.826.0.1.3680043.32458']
1.2.826.0.1.3680043.30020: isdir=True, isfile=False, size=N/A
1.2.826.0.1.3680043.4740: isdir=True, isfile=False, size=N/A
1.2.826.0.1.3680043.30640: isdir=True, isfile=False, size=N/A
1.2.826.0.1.3680043.5581: isdir=True, isfile=False
1.2.826.0.1.3680043.7479: isdir=True, isfile=False
1.2.826.0.1.3680043.26665: isdir=True, isfile=False
